In [528]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import random
import collections
import pandas as pd
import csv

# Load Table, Check and Remove Duplications

In [2]:
with open('DellGSPE_DSrole_dataset.csv','r') as f:
    inp_raw = pd.read_csv(f)

In [4]:
pd.get_option("display.max_columns")

20

In [239]:
inp_raw

,asst_id,product_type,mnfture_wk,contract_st,contract_end,contact_wk,contact_type,issue_type,topic_category,parts_sent,repair_type,repeat_ct,parts_ct,agent_tenure_indays,contact_manager_flg,diagnostics,repeat_parts_sent,region,country
0,0,Laptops,201726,201726,202125,201840,Voice,NaN,NaN,"Hard Drive, Documentation(2)",Hard,0,3,1018.0,0,Not_USED,NaN,Hogwarts,NaN
1,1,Laptops,201650,201650,201949,201840,Voice,NaN,NaN,"Speaker, Chassis",Hard,0,2,1018.0,0,Not_USED,NaN,Hogwarts,Zonko's Joke Shop
2,2,Laptops,201821,201821,202222,201840,Voice,NaN,NaN,Cables,Hard,0,1,1018.0,0,Not_USED,"Motherboard, Miscellaneous(2), Hard Drive, Cha...",Hogwarts,Zonko's Joke Shop
3,3,Laptops,201608,201608,201908,201840,VOICE,Hard Drive,Booting,Hard Drive,Hard,0,1,298.0,0,Effective_Usage,NaN,Hogwarts,Zonko's Joke Shop
4,4,Laptops,201732,201732,202032,201840,Voice,NaN,NaN,Battery (Notebook),Hard,0,1,1018.0,0,Not_USED,NaN,Hogwarts,Zonko's Joke Shop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,16660,Laptops,201630,201630,201930,201840,VOICE,Information Status,Booting,NaN,Soft,0,0,1978.0,0,Not_USED,NaN,Hogwarts,Zonko's Joke Shop
99996,16661,Laptops,201718,201718,202122,201840,Voice,NaN,NaN,"LCD Panel, Chassis(2)",Hard,0,3,1018.0,0,Not_USED,Chassis,Hogwarts,Zonko's Joke Shop
99997,16662,Desktops,201607,201607,202107,201840,VOICE,Information Status,Power,NaN,Soft,0,0,305.0,0,Not_USED,NaN,Milky Way,Merope
99998,16663,Laptops,201552,201552,202352,201840,CHAT,"Audio, Video, Speakers, TV Tuner",LCD/Monitor,"LCD Panel, Cables",Hard,0,2,323.0,0,InEffective_Usage,NaN,Hogwarts,Zonko's Joke Shop


In [9]:
# Check Data Type
inp_raw.dtypes

asst_id                  int64
product_type            object
mnfture_wk               int64
contract_st              int64
contract_end             int64
contact_wk               int64
contact_type            object
issue_type              object
topic_category          object
parts_sent              object
repair_type             object
repeat_ct                int64
parts_ct                 int64
agent_tenure_indays    float64
contact_manager_flg      int64
diagnostics             object
repeat_parts_sent       object
region                  object
country                 object
dtype: object

In [574]:
#Numerical Data Statistics
inp_raw.describe()

,asst_id,mnfture_wk,contract_st,contract_end,contact_wk,repeat_ct,parts_ct,agent_tenure_indays,contact_manager_flg
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.0,100000.000000,100000.00000,99986.000000,100000.000000
mean,35702.117890,201672.108910,201671.595910,201951.897740,201840.0,0.074090,1.15955,985.886844,0.012430
std,25038.092288,156.139692,158.940598,186.777325,0.0,0.262491,1.56942,862.044503,0.110795
min,0.000000,201001.000000,200547.000000,201006.000000,201840.0,0.000000,0.00000,218.000000,0.000000
25%,12382.750000,201612.000000,201612.000000,201903.000000,201840.0,0.000000,0.00000,536.000000,0.000000
50%,32898.500000,201719.000000,201720.000000,201938.000000,201840.0,0.000000,1.00000,801.000000,0.000000
75%,57701.250000,201808.000000,201809.000000,202045.000000,201840.0,0.000000,2.00000,1018.000000,0.000000
max,82441.000000,201927.000000,202630.000000,202953.000000,201840.0,3.000000,51.00000,8922.000000,1.000000


In [241]:
# Check Unique assistant ID
inp_raw['asst_id'].unique().shape

(82442,)

In [249]:
# Check repetition of assistant ID
# So as to check potential duplication
collections.Counter(inp_raw.groupby(['asst_id'])['asst_id'].count())

Counter({2: 17026, 4: 140, 18: 1, 14: 1, 6: 1, 1: 65258, 7: 1, 3: 13, 46: 1})

In [583]:
# Drop duplicated records
df_edit = inp_raw.drop_duplicates()

In [256]:
# Recheck the repetition of assistant ID after removing duplicated records
collections.Counter(df_edit.groupby(['asst_id'])['asst_id'].count())

Counter({2: 3891, 1: 78480, 3: 63, 10: 1, 8: 1, 4: 4, 7: 1, 44: 1})

In [272]:
# Need to check the records with repetition of 2
# For the records with repetition larger than 3, since the ratio is small, may need to handle them individually if neccessary
# If any modeling is needed, may just drop those rare records with large repetition.

#generate my repetition qc table 

reqc = pd.DataFrame(df_edit.groupby('asst_id')['asst_id'].count())
reqc = reqc.rename({'asst_id':'repetition'},axis =1)

In [288]:
# Merge the repetion number into data frame for QC:
df_edit = df_edit.merge(reqc,how='left',on='asst_id')

In [286]:
# extract records with repetition of 2
df_edit[df_edit['repetition']==2].sort_values('asst_id')

,asst_id,product_type,mnfture_wk,contract_st,contract_end,contact_wk,contact_type,issue_type,topic_category,parts_sent,repair_type,repeat_ct,parts_ct,agent_tenure_indays,contact_manager_flg,diagnostics,repeat_parts_sent,region,country,repetition
0,0,Laptops,201726,201726,202125,201840,Voice,NaN,NaN,"Hard Drive, Documentation(2)",Hard,0,3,1018.0,0,Not_USED,NaN,Hogwarts,NaN,2
83175,0,Laptops,201726,201726,202125,201840,Voice,NaN,NaN,"Hard Drive, Documentation(2)",Hard,0,3,1018.0,0,Not_USED,NaN,Hogwarts,Zonko's Joke Shop,2
83176,8,Desktops,201821,201821,202122,201840,VOICE,Backup / Recovery Software,OS,NaN,Soft,0,0,2055.0,0,Not_USED,NaN,Middle Earth,NaN,2
8,8,Desktops,201821,201821,202122,201840,VOICE,Backup / Recovery Software,OS,NaN,Soft,0,0,2055.0,0,Not_USED,NaN,Middle Earth,Lorien,2
10,10,Desktops,201644,201644,201744,201840,VOICE,Contract or Service Issue,NaN,NaN,Hard,0,0,606.0,0,Not_USED,NaN,Milky Way,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82928,82197,NaN,201830,201830,201943,201840,VOICE,Information Request,NaN,NaN,Soft,0,0,473.0,0,Not_USED,NaN,Middle Earth,Lorien,2
82975,82244,Laptops,201625,201625,202025,201840,CHAT,Customer Confirmed CID,Physical Damage,NaN,Hard,0,0,1849.0,0,Not_USED,NaN,Middle Earth,Lorien,2
82976,82244,Laptops,201625,201625,202025,201840,VOICE,Input Device / Webcam,Call Logs,Keyboard,Hard,0,1,463.0,0,Not_USED,NaN,Middle Earth,Lorien,2
83149,82416,Laptops,201706,201706,202006,201840,Voice,NaN,NaN,LCD,Hard,1,1,1018.0,0,Not_USED,Chassis,Hogwarts,Zonko's Joke Shop,2


In [311]:
# create a ID Country data frame

df_country = inp_raw[['asst_id','country']].dropna(subset = ['country']).drop_duplicates().sort_values('asst_id')
df_country

,asst_id,country
83182,0,Zonko's Joke Shop
1,1,Zonko's Joke Shop
2,2,Zonko's Joke Shop
3,3,Zonko's Joke Shop
4,4,Zonko's Joke Shop
...,...,...
83176,82436,Zonko's Joke Shop
83177,82437,Zonko's Joke Shop
83178,82438,Lorien
83179,82439,Zonko's Joke Shop


In [314]:
# A saver assumption is that one case should have only 1 region/country
# So try to drop the country/region to recheck the duplications
# I drop only the country and both region + country and the results seems to be the same, so just drop the country.
# The number of instances (originally repetion == 2) reduced from 7782 to 4476

# drop country and then remove duplicate one more time
# merge the country column back after revmoving duplicates

df_edit = df_edit.drop('country', axis = 1).drop_duplicates().sort_values('asst_id')

In [318]:
df_edit = df_edit.merge(df_country, how = 'left', on = 'asst_id')

In [343]:
df_edit

,asst_id,product_type,mnfture_wk,contract_st,contract_end,contact_wk,contact_type,issue_type,topic_category,parts_sent,repair_type,repeat_ct,parts_ct,agent_tenure_indays,contact_manager_flg,diagnostics,repeat_parts_sent,region,country
0,0,Laptops,201726,201726,202125,201840,Voice,NaN,NaN,"Hard Drive, Documentation(2)",Hard,0,3,1018.0,0,Not_USED,NaN,Hogwarts,Zonko's Joke Shop
1,1,Laptops,201650,201650,201949,201840,Voice,NaN,NaN,"Speaker, Chassis",Hard,0,2,1018.0,0,Not_USED,NaN,Hogwarts,Zonko's Joke Shop
2,2,Laptops,201821,201821,202222,201840,Voice,NaN,NaN,Cables,Hard,0,1,1018.0,0,Not_USED,"Motherboard, Miscellaneous(2), Hard Drive, Cha...",Hogwarts,Zonko's Joke Shop
3,3,Laptops,201608,201608,201908,201840,VOICE,Hard Drive,Booting,Hard Drive,Hard,0,1,298.0,0,Effective_Usage,NaN,Hogwarts,Zonko's Joke Shop
4,4,Laptops,201732,201732,202032,201840,Voice,NaN,NaN,Battery (Notebook),Hard,0,1,1018.0,0,Not_USED,NaN,Hogwarts,Zonko's Joke Shop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83167,82437,Laptops,201751,201750,202050,201840,VOICE,System Board Components,NaN,"Motherboard, Documentation, Chassis",Hard,0,3,361.0,0,InEffective_Usage,NaN,Hogwarts,Zonko's Joke Shop
83168,82438,Laptops,201832,201832,201946,201840,VOICE,Information Status,Booting,NaN,Soft,0,0,325.0,0,Not_USED,NaN,Middle Earth,Lorien
83169,82439,Laptops,201710,201710,201809,201840,VOICE,System Board Components,NaN,"Miscellaneous(2), Chassis, Cables",Hard,0,4,1901.0,0,Not_USED,NaN,Hogwarts,Zonko's Joke Shop
83170,82440,Laptops,201837,201721,201917,201840,VOICE,Operating System,Other,NaN,Soft,0,0,541.0,0,Not_USED,NaN,Hogwarts,NaN


In [344]:
# recheck the repetitions.
# seems quite some duplication due to country

collections.Counter(df_edit.groupby(['asst_id'])['asst_id'].count())

Counter({1: 81786, 2: 637, 8: 1, 6: 1, 7: 1, 3: 12, 4: 3, 43: 1})

In [339]:
# drop the original repetion information and update it
df_edit = df_edit.drop('repetition', axis = 1)

In [348]:
rpqc1 = pd.DataFrame(df_edit.groupby('asst_id')['asst_id'].count())
rpqc1 = rpqc1.rename({'asst_id':'repetition'},axis =1)

In [349]:
rpqc1

,repetition
asst_id,
0,1
1,1
2,1
3,1
4,1
...,...
82437,1
82438,1
82439,1


In [350]:
# merge the updated repetition to the table
df_edit = df_edit.merge(rpqc1,how = 'left', on = 'asst_id')

# Explore the data and QC data quality

In [399]:
# Check counts and data type just for convenience
pd.concat([df_edit.count(),df_edit.dtypes],axis = 1)

,0,1
asst_id,83172,int64
product_type,83152,object
mnfture_wk,83172,int64
contract_st,83172,int64
contract_end,83172,int64
contact_wk,83172,int64
contact_type,83163,object
issue_type,71488,object
topic_category,55692,object
parts_sent,45354,object


# Explore the Categorical Data

In [415]:
# Extrac a list for Categorical columns
d_type = df_edit.dtypes
cat_list = list(d_type[d_type == 'object'].index)
cat_list

['product_type',
 'contact_type',
 'issue_type',
 'topic_category',
 'parts_sent',
 'repair_type',
 'diagnostics',
 'repeat_parts_sent',
 'region',
 'country']

In [423]:
# Unify the case
df_edit['contact_type'] = df_edit['contact_type'].str.upper()

In [427]:
# Replace the Null values into UNKOWN for the contact type
df_edit = df_edit.fillna(value={'contact_type':'UNKNOWN'})

In [572]:
# Chcek frequency of inidiviual categorical columns

for i in cat_list:
    print('Number of categories for \"',i,"\" :",df_edit[i].value_counts().count())
    print(df_edit[i].value_counts(dropna=False))
    print()

Number of categories for " product_type " : 3
Laptops              57444
Desktops             18719
Other Electronics     2434
NaN                     15
Name: product_type, dtype: int64

Number of categories for " contact_type " : 4
VOICE      66170
CHAT       11304
EMAIL       1044
UNKNOWN       94
Name: contact_type, dtype: int64

Number of categories for " issue_type " : 80
NaN                               11358
System Board Components           10188
Fee Based Support                  8177
Operating System                   7550
Hard Drive                         6759
                                  ...  
Shipping Error / Factory Error        1
Compellent - Hardware                 1
Lost / Stolen in Transit              1
Order Tracking                        1
MWD                                   1
Name: issue_type, Length: 81, dtype: int64

Number of categories for " topic_category " : 63
NaN                     25983
Booting                 12095
LCD/Monitor              4

# Explore the Continuous Data

In [500]:
#Approximate week function
#Ignore the case that the number of weeks will be different from year to year
def diffwk(w):
    return w//100 * 52 + w%100

# Check negative values for certain time span
def diffwkqc(str1:str,str2:str,qcvalues = True):
    print('negative value count:',df_edit[df_edit[str1].apply(diffwk) - df_edit[str2].apply(diffwk) < 0].shape[0])
    
    if qcvalues:
        for i in sorted(set(collections.Counter(df_edit[str1].apply(diffwk) - df_edit[str2].apply(diffwk)).keys())):
            print(i, end=' ')        

In [501]:
# Check warrenty end week vs. start week
diffwkqc('contract_end','contract_st',False)

negative value count: 0


In [502]:
# Check warrenty start time vs. manufacturing week
# found 3861 records that the warrenty started week earlier than manufacturing week
# Does it make senese?

diffwkqc('contract_st','mnfture_wk')

negative value count: 3861
-524 -493 -484 -458 -456 -447 -431 -372 -371 -369 -361 -358 -352 -334 -327 -313 -291 -288 -272 -269 -265 -262 -257 -253 -252 -250 -249 -248 -243 -240 -233 -232 -230 -227 -226 -225 -222 -221 -219 -215 -214 -213 -212 -210 -208 -207 -206 -205 -204 -203 -202 -201 -199 -197 -195 -194 -193 -191 -190 -189 -186 -184 -183 -182 -181 -180 -179 -177 -173 -172 -170 -169 -168 -167 -166 -165 -163 -162 -161 -160 -159 -158 -157 -156 -155 -154 -153 -152 -151 -150 -149 -148 -147 -146 -145 -144 -143 -142 -141 -140 -139 -138 -137 -136 -135 -134 -133 -132 -131 -130 -129 -128 -127 -126 -125 -124 -123 -122 -121 -120 -119 -118 -117 -116 -115 -114 -113 -112 -111 -109 -108 -107 -106 -105 -104 -103 -102 -101 -100 -98 -97 -96 -95 -94 -93 -92 -91 -90 -89 -88 -87 -86 -85 -84 -83 -82 -81 -80 -79 -78 -77 -76 -75 -74 -73 -72 -71 -70 -69 -68 -67 -66 -65 -64 -63 -62 -61 -60 -59 -58 -57 -56 -55 -54 -53 -52 -51 -50 -49 -48 -47 -46 -45 -44 -43 -42 -41 -40 -39 -38 -37 -36 -35 -34 -33 -32 -31 -30 -2

In [507]:
#Check some extreme cases
df_edit[df_edit['contract_st'].apply(diffwk) - df_edit['mnfture_wk'].apply(diffwk) == -524]
# Should be a typo on the contract start year

,asst_id,product_type,mnfture_wk,contract_st,contract_end,contact_wk,contact_type,issue_type,topic_category,parts_sent,repair_type,repeat_ct,parts_ct,agent_tenure_indays,contact_manager_flg,diagnostics,repeat_parts_sent,region,country,repetition
27668,27432,Desktops,201551,200547,202051,201840,VOICE,System Board Components,Booting,"Motherboard, Documentation",Hard,0,2,863.0,0,Effective_Usage,NaN,Hogwarts,Zonko's Joke Shop,1


In [504]:
# Check contact week vs warrenty start week
# Does it make sense the customer contacted before warrentty start?

diffwkqc('contact_wk','contract_st')

negative value count: 729
-406 -297 -39 -38 -37 -36 -35 -34 -33 -32 -31 -30 -29 -28 -27 -26 -25 -24 -23 -22 -21 -20 -19 -18 -17 -16 -15 -14 -13 -12 -11 -10 -9 -8 -7 -6 -5 -4 -3 -2 -1 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231

In [517]:
#Check some extreme cases
df_edit[df_edit['contact_wk'].apply(diffwk) - df_edit['contract_st'].apply(diffwk) == -406]

,asst_id,product_type,mnfture_wk,contract_st,contract_end,contact_wk,contact_type,issue_type,topic_category,parts_sent,repair_type,repeat_ct,parts_ct,agent_tenure_indays,contact_manager_flg,diagnostics,repeat_parts_sent,region,country,repetition
44516,44150,Laptops,201530,202630,202630,201840,VOICE,NaN,NaN,Motherboard,Hard,0,1,1018.0,0,Not_USED,NaN,Hogwarts,Zonko's Joke Shop,1


In [505]:
# Check contact week vs manufacturing week
# Do it make sense the customer contacted before manufacuring?

diffwkqc('contact_wk','mnfture_wk')

negative value count: 1243
-39 -38 -37 -36 -35 -34 -33 -32 -31 -30 -29 -28 -27 -26 -25 -24 -23 -22 -21 -20 -19 -18 -17 -16 -15 -14 -13 -12 -11 -10 -9 -8 -7 -6 -5 -4 -3 -2 -1 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 

In [515]:
#Check some extreme cases
#Problematic Record. May just drop it
df_edit[df_edit['contact_wk'].apply(diffwk) - df_edit['mnfture_wk'].apply(diffwk) == -39]

,asst_id,product_type,mnfture_wk,contract_st,contract_end,contact_wk,contact_type,issue_type,topic_category,parts_sent,repair_type,repeat_ct,parts_ct,agent_tenure_indays,contact_manager_flg,diagnostics,repeat_parts_sent,region,country,repetition
14252,14120,Desktops,201927,201717,202117,201840,VOICE,Operating System,Booting,NaN,Soft,0,0,907.0,0,Not_USED,NaN,Hogwarts,Zonko's Joke Shop,1
19028,18860,Laptops,201927,201749,202027,201840,VOICE,Hard Drive,Booting,"Unassigned, Hard Drive",Hard,0,2,774.0,0,Not_USED,NaN,Middle Earth,Erebor,1
26938,26711,Laptops,201927,201927,202228,201840,VOICE,Operating System,OS,NaN,Soft,0,0,424.0,0,Not_USED,NaN,Hogwarts,Zonko's Joke Shop,1
35374,35075,Laptops,201927,201840,202140,201840,VOICE,Hard Drive,LCD/Monitor,Miscellaneous(2),Hard,0,2,424.0,0,Not_USED,NaN,Hogwarts,Zonko's Joke Shop,1
43181,42820,Laptops,201927,201833,202027,201840,CHAT,Information Status,System Performance,NaN,Soft,0,0,462.0,0,Not_USED,NaN,Middle Earth,Helm's Deep,1
49516,49106,Laptops,201927,201927,202027,201840,VOICE,Operating System,NaN,NaN,Soft,0,0,396.0,0,Not_USED,NaN,Middle Earth,Helm's Deep,1
51253,50832,Other Electronics,201927,201734,202034,201840,CHAT,Imaging Device,NaN,Base Unit,Hard,0,1,807.0,0,Not_USED,Base Unit,Hogwarts,Zonko's Joke Shop,1
51838,51411,Laptops,201927,201927,202027,201840,VOICE,Operating System,System Performance,NaN,Soft,0,0,354.0,0,Not_USED,NaN,Middle Earth,Helm's Deep,1
56982,56521,Other Electronics,201927,201809,201910,201840,VOICE,Hard Drive,Booting,"Miscellaneous, Hard Drive",Hard,0,2,720.0,0,Effective_Usage,NaN,Hogwarts,Zonko's Joke Shop,1
57576,57105,Laptops,201927,201829,202230,201840,VOICE,Information Status,NaN,NaN,Soft,0,0,1796.0,0,Not_USED,NaN,Milky Way,Merope,1


In [521]:
# Check the agent tenure indays
df_edit['agent_tenure_indays'].max(), df_edit['agent_tenure_indays'].min()

(8922.0, 218.0)

In [527]:
# Check the contact manger flag
df_edit.groupby(['contact_manager_flg'])['contact_manager_flg'].count()

contact_manager_flg
0    82147
1     1025
Name: contact_manager_flg, dtype: int64

In [541]:
#Calulate warrenty duration by weeks
df_edit['contract_dur_wk'] = df_edit['contract_end'].apply(diffwk) - df_edit['contract_st'].apply(diffwk)

In [542]:
#Calculate the time from manufacturing to warrenty start
df_edit['mnf_contract_wk'] = df_edit['contract_st'].apply(diffwk) - df_edit['mnfture_wk'].apply(diffwk)

In [543]:
#Calculate how many weeks the client contacts agents after warrenty start
df_edit['contact_contract_wk'] = df_edit['contact_wk'].apply(diffwk) - df_edit['contract_st'].apply(diffwk)

In [546]:
# Drop the columns that manufacturing week is earlier than the warrenty start week
# Drop the columns that client contacted agents earlier than warrenty started

df_edit = df_edit[~df_edit['mnf_contract_wk']<0][~df_edit['contact_contract_wk']<0]

C:\Users\pzhen\Machine_Learning\Anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [558]:
df_edit

,asst_id,product_type,mnfture_wk,contract_st,contract_end,contact_wk,contact_type,issue_type,topic_category,parts_sent,...,agent_tenure_indays,contact_manager_flg,diagnostics,repeat_parts_sent,region,country,repetition,contract_dur_wk,mnf_contract_wk,contact_contract_wk
0,0,Laptops,201726,201726,202125,201840,VOICE,NaN,NaN,"Hard Drive, Documentation(2)",...,1018.0,0,Not_USED,NaN,Hogwarts,Zonko's Joke Shop,1,207,0,66
1,1,Laptops,201650,201650,201949,201840,VOICE,NaN,NaN,"Speaker, Chassis",...,1018.0,0,Not_USED,NaN,Hogwarts,Zonko's Joke Shop,1,155,0,94
2,2,Laptops,201821,201821,202222,201840,VOICE,NaN,NaN,Cables,...,1018.0,0,Not_USED,"Motherboard, Miscellaneous(2), Hard Drive, Cha...",Hogwarts,Zonko's Joke Shop,1,209,0,19
3,3,Laptops,201608,201608,201908,201840,VOICE,Hard Drive,Booting,Hard Drive,...,298.0,0,Effective_Usage,NaN,Hogwarts,Zonko's Joke Shop,1,156,0,136
4,4,Laptops,201732,201732,202032,201840,VOICE,NaN,NaN,Battery (Notebook),...,1018.0,0,Not_USED,NaN,Hogwarts,Zonko's Joke Shop,1,156,0,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83165,82435,Laptops,201825,201825,202126,201840,VOICE,System Board Components,Booting,"Motherboard, Miscellaneous(2), Documentation",...,720.0,1,Effective_Usage,NaN,Hogwarts,Zonko's Joke Shop,1,157,0,15
83166,82436,Laptops,201652,201652,201952,201840,CHAT,System Board Components,NaN,"Motherboard, Miscellaneous(2), Heatsink, Fan A...",...,837.0,0,Not_USED,NaN,Hogwarts,Zonko's Joke Shop,1,156,0,92
83168,82438,Laptops,201832,201832,201946,201840,VOICE,Information Status,Booting,NaN,...,325.0,0,Not_USED,NaN,Middle Earth,Lorien,1,66,0,8
83169,82439,Laptops,201710,201710,201809,201840,VOICE,System Board Components,NaN,"Miscellaneous(2), Chassis, Cables",...,1901.0,0,Not_USED,NaN,Hogwarts,Zonko's Joke Shop,1,51,0,82


In [555]:
# check min max of client reaching out time
df_edit['contact_contract_wk'].max(),df_edit['contact_contract_wk'].min()

(455, 0)

In [556]:
# check min max of time the warrenty started after manufactured
df_edit['mnf_contract_wk'].max(),df_edit['mnf_contract_wk'].min()

(276, 0)

In [557]:
# check min max of warrenty duration
df_edit['contract_dur_wk'].max(),df_edit['contract_dur_wk'].min()

(961, 12)

In [560]:
# Check the categorical data one more time after cleaning the time data

d_type = df_edit.dtypes
cat_list = list(d_type[d_type == 'object'].index)
cat_list

for i in cat_list:
    print('Number of categories for \"',i,"\" :",df_edit[i].value_counts().count())
    print(df_edit[i].value_counts(dropna=False))
    print()

Number of categories for " product_type " : 3
Laptops              57444
Desktops             18719
Other Electronics     2434
NaN                     15
Name: product_type, dtype: int64

Number of categories for " contact_type " : 4
VOICE      66170
CHAT       11304
EMAIL       1044
UNKNOWN       94
Name: contact_type, dtype: int64

Number of categories for " issue_type " : 80
NaN                               11358
System Board Components           10188
Fee Based Support                  8177
Operating System                   7550
Hard Drive                         6759
                                  ...  
Shipping Error / Factory Error        1
Compellent - Hardware                 1
Lost / Stolen in Transit              1
Order Tracking                        1
MWD                                   1
Name: issue_type, Length: 81, dtype: int64

Number of categories for " topic_category " : 63
NaN                     25983
Booting                 12095
LCD/Monitor              4

In [566]:
#Save the data for analysis
df_edit.to_csv('data_bk2_af_drop_time.csv', header='fieldname', line_terminator='\n', index=False)